In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="2oeGI3QWbiYo6HNrErIp")
project = rf.workspace("aaaa-g4pgj").project("my-first-project-pv3aw")
version = project.version(11)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [2]:
import cv2, numpy as np

def specular_mask_soft(bgr, p=97, s_max=80, nearwhite=210):
    # HSV gates (bright & low-sat)
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    h,s,v = cv2.split(hsv)

    # adaptive brightness threshold (percentile of V)
    thr_v = np.percentile(v, p)
    bright = (v >= thr_v)

    # low saturation typical for speculars
    low_sat = (s <= s_max)

    # near-white in RGB to avoid killing colored objects
    b,g,r = cv2.split(bgr)
    near_w = (r > nearwhite) & (g > nearwhite) & (b > nearwhite)
    close_channels = (np.abs(r-g) < 18) & (np.abs(r-b) < 18) & (np.abs(g-b) < 18)

    m = (bright & low_sat) | (near_w & close_channels)
    m = m.astype(np.uint8) * 255

    # clean up mask gently
    m = cv2.morphologyEx(m, cv2.MORPH_OPEN, np.ones((3,3),np.uint8), iterations=1)
    m = cv2.GaussianBlur(m, (5,5), 0)
    return m

def deglare_conservative(bgr, alpha=0.65, bilateral_d=7, bilateral_sc=60, bilateral_ss=60,
                         clahe_clip=1.2):
    m = specular_mask_soft(bgr)

    # make a smoothed version (no hallucination like inpainting)
    smooth = cv2.bilateralFilter(bgr, d=bilateral_d, sigmaColor=bilateral_sc, sigmaSpace=bilateral_ss)

    # soft blend only inside mask
    m_f = (m.astype(np.float32)/255.0)[...,None]
    out = (bgr.astype(np.float32)*(1-m_f) + (alpha*smooth + (1-alpha)*bgr).astype(np.float32)*m_f).astype(np.uint8)

    # very light global touch (optional): CLAHE on L only
    lab = cv2.cvtColor(out, cv2.COLOR_BGR2LAB)
    L,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clahe_clip, tileGridSize=(8,8))
    L2 = clahe.apply(L)
    out = cv2.cvtColor(cv2.merge([L2,a,b]), cv2.COLOR_LAB2BGR)

    return out